In [ ]:
import pandas as pd
import json
import numpy as np
from ast import literal_eval
import scipy.io as sio
from scipy.stats import norm
from datetime import datetime, timedelta
#SQL
import psycopg2 as pg2
#Plots 
import matplotlib.pyplot as plt
import seaborn as sns

#Helpers
from pre_processing import *

#Others
import warnings
warnings.filterwarnings('ignore')

[1) MIMIC_WAVEFORM_DATABASE_SQL](#MIMIC_WAVEFORM_DATABASE_SQL)

* **df_filtered_signals**

[2) MIMIC_CLINICAL_DATABASE_SQL](#MIMIC_CLINICAL_DATABASE_SQL)

[2.a)ICU + HADM](#icu_hadm)

* **df_filtered_signals_icu_hadm**

[2.a) Age + Gender](#age_sex)

* **df_filtered_signals_icu_hadm_age_sex**

<a id='MIMIC_WAVEFORM_DATABASE_SQL'></a>

# MIMIC WAVEFORM DATABASE SQL

Use the mimic.pg_dump file to load the postgres database using psql database_name < mimic.pg_dump
You can then query waveforms metadata in the notebook.

**Thanks to @Dubrzr to share the Postgre-SQL for MIMIC-WFDB**
Repository: https://github.com/Dubrzr/mimic3-scripts/tree/master/mimic3wdb-matched-metadata


In [ ]:
parametros={}

In [ ]:
info_columns = ['adc_gain', 'adc_res', 'adc_zero', 'baseline',
       'block_size', 'comments', 'date_reliability', 'enddate', 'fmt',
       'frequency', 'init_vals', 'length_ms', 'pxx', 'pxxxxxx', 'record',
       'signals', 'startdate', 'subject_id', 'subrecord', 'type']

In [ ]:
info_columns_drop = ['adc_gain', 'adc_res', 'adc_zero', 'baseline',
       'block_size','fmt', 'init_vals','type','frequency']

In [ ]:
signals_colums = ['%','ABP', 'ABPDias', 'ABPMean', 'ABPSys', 'AOBP', 'ART', 'AVF', 'AVL',
       'AVR', 'CO', 'CVP', 'CVP_1/3', 'CVP_2/3', 'CVP_3/3', 'HR', 'I', 'ICP',
       'II', 'III', 'LAP', 'MCL', 'MCL1', 'NBP', 'NBPDias', 'NBPMean',
       'NBPSys', 'P1', 'PAP', 'PAPDias', 'PAPMean', 'PAPSys', 'PAWP', 'PLETH',
       'PULSE', 'PVC', 'RAP', 'RAP_1/3', 'RAP_2/3', 'RAP_3/3', 'RESP', 'ST',
       'ST_I', 'ST_II', 'UAP', 'UVP', 'V', 'V1', 'V2', 'dSpO2']

In [ ]:
comorbidities_colums =['congestive_heart_failure', 'cardiac_arrhythmias','valvular_disease',
               'pulmonary_circulation', 'peripheral_vascular','hypertension','paralysis',
               'other_neurological','chronic_pulmonary','diabetes_uncomplicated',
               'diabetes_complicated','hypothyroidism','renal_failure','liver_disease',
               'peptic_ulcer','aids','lymphoma','metastatic_cancer','solid_tumor','rheumatoid_arthritis',
               'coagulopathy','obesity','weight_loss','fluid_electrolyte','blood_loss_anemia',
               'deficiency_anemias','alcohol_abuse','drug_abuse','psychoses','depression']

In [ ]:
parametros['info_columns'] = info_columns
parametros['signals_colums'] = signals_colums
parametros['comorbidities_colums'] = comorbidities_colums

In [ ]:
plot_path = 'figures/ABP_PPG/'
parametros['plot_path'] = plot_path

In [ ]:
#SQL Connection
parametros['host'] = "localhost"
parametros['database'] = "mimic_wfdb"
parametros['user'] = "mimic"
parametros['password'] = "xxxx"

# SQL Query
# Canales Seleccionados

parametros['channels_selected_AND'] = ['PLETH'] #first 'n' channel_selected
parametros['channels_selected_OR'] =['ABP','ART'] #this represent the lastone channel_selected

parametros['channels_selected'] = ['PLETH','ABP'] #MANUALLY ADJUSTED [first AND channel, then OR]

# Limite para borrar columa de NaN
parametros['drop_nan_limit'] = 1
# Configuracion de las señales
parametros['fs'] = 125
parametros['q_signal'] =  2
parametros['ventana'] =  15    #segundos
parametros['intervalo'] =  5   # minutos
parametros['signal_i'] =  10   # minutos / otro criterio, discutir otros criterios
parametros['minutos_min'] = parametros['signal_i'] + ( parametros['q_signal'] -1 )*parametros['intervalo']

#Clincal Filters
parametros['age_min'] = 18 #Min age
parametros['age_max'] = 90 #Min age

parametros['weight_min'] = 40
parametros['weight_max'] = 180
parametros['threshold_day'] = 2 # Used to alocate the icustay_id

#Processing Thresholds [mmHg]
parametros['DBP_low'] = 50
parametros['SBP_low'] = 80
parametros['DBP_high'] = 120
parametros['SBP_high'] = 200
parametros['delta_DBP_valley'] = 7
parametros['delta_SBP_peak'] = 30

In [ ]:
df_filtered = func_sql_filtered(parametros)

In [ ]:
print(f'Cantidad de Registros: {df_filtered.shape[0]}')
print('Cantidad de Sujetos: {}'.format(np.shape(df_filtered['subject_id'].unique())[0]))

# Saved Pre-Processing

In [ ]:
"""json = json.dumps(parametros)
f = open("../df_filtered/ABP_PPG/parametros.json","w")
f.write(json)
f.close()
sio.savemat('../df_filtered/ABP_PPG/parametros.mat',parametros)
df_filtered['startdate'] =  pd.to_datetime(df_filtered['startdate']).dt.date
df_filtered = df_filtered.drop(columns=info_columns_drop)
df_filtered.to_csv('../df_filtered/ABP_PPG/df_filtered_signals.csv')
"""

# MIMIC CLINICAL DATABASE SQL

* PSYCOPG2

* Pandas (sql)    

# Connection

In [ ]:
#Conection
conn = pg2.connect(
host = "localhost",
database = "mimic",
user = "postgres",
password = "postgres")
#Cursor
cursor = conn.cursor()
#Adjust to the schema
path2schema = "SET search_path TO mimiciii;"
cursor.execute(path2schema)

<a id='icu_hadm'></a>

# Querys

## ICU_ID and HADM_ID

In [ ]:
# 1) icu_id /hadm_id --> based on ICUSTAYS table FROM mimic
#Usamos el subject_id en formato "tuple" para poder usarlo con el IN del query

ids_sql = tuple(df_filtered['subject_id'].values.astype(int).tolist())

text_sql = """
SELECT *
FROM mimiciii.icustays AS I 
WHERE I.subject_id IN %s ORDER BY I.subject_id ASC;
"""
query_sql = cursor.mogrify(text_sql,(ids_sql,))

In [ ]:
icu_hadm_df = pd.read_sql_query(query_sql, conn)
icu_hadm_df = icu_hadm_df.rename(columns={"intime": "startdate"})
icu_hadm_df['startdate'] =  pd.to_datetime(icu_hadm_df['startdate']).dt.date
print(icu_hadm_df.shape)

In [ ]:
df_filtered['icustay_id'] = np.nan
df_filtered['hadm_id'] = np.nan

for index, row in df_filtered.iterrows():
    subject_id = row.subject_id
    matches = icu_hadm_df.loc[(icu_hadm_df.subject_id == row.subject_id)]
    if not matches.empty:
        if (index % 1000)==0:
            print(index,'Completed')
        #Diference in days between record and all de icu entries
        matches['deltas'] = (row.startdate - matches.startdate) / np.timedelta64(1, 'D') #to float
        #Only between [ threshold < delta < LOS+threshold ]
        matches = matches.loc[(-1*parametros['threshold_day']<= matches['deltas']) & (matches['deltas']<=np.ceil(matches['los'])+parametros['threshold_day'])]
        #absolute difference
        matches['deltas'] = matches['deltas'].abs()
        if not matches.empty:
            #min delta difference
            d_idx = matches.deltas.idxmin()
            #assing icustay_id & hadm_id to record (df_filtered)
            df_filtered.loc[index,['icustay_id','hadm_id']] = matches.loc[d_idx,['icustay_id','hadm_id']]

In [ ]:
#NaN summarize
print(f'Droping NaNs...')
total = df_filtered.isnull().sum().sort_values(ascending=False)
percent = (df_filtered.isnull().sum()/df_filtered.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data)

In [ ]:
#Drop Nan
df_filtered = df_filtered.dropna()
print(df_filtered.shape)

In [ ]:
df_filtered.icustay_id = df_filtered.icustay_id.astype(int)
df_filtered.hadm_id = df_filtered.hadm_id.astype(int)

In [ ]:
print(f'Cantidad de Registros: {df_filtered.shape[0]}')
print('Cantidad de Sujetos: {}'.format(np.shape(df_filtered['subject_id'].unique())[0]))

In [ ]:
#df_filtered.to_csv('../df_filtered/ABP_PPG/df_filtered_signals_icu_hadm.csv')

<a id='age_sex'></a>

## Age and Gender

In [ ]:
# 2)age, gender from --> icu_details
icu_ids_sql = tuple(df_filtered['icustay_id'].values.astype(int).tolist())

text_sql = """
SELECT ID.icustay_id , ID.age, ID.gender,ID.los_hospital, ID.los_icu
FROM mimiciii.icustay_detail AS ID 
WHERE ID.icustay_id IN %s;
"""
query_sql = cursor.mogrify(text_sql,(icu_ids_sql,))

In [ ]:
sex_dob_df = pd.read_sql_query(query_sql, conn)
print(sex_dob_df.shape)

In [ ]:
df_filtered = df_filtered.merge(sex_dob_df,how='inner',on='icustay_id')
df_filtered.shape

In [ ]:
df_filtered = df_filtered.loc[(df_filtered['age']>=parametros['age_min'])\
                              &(df_filtered['age']<=parametros['age_max'])]
print(df_filtered.shape)

In [ ]:
n_bins = 70
figure = plt.figure(figsize=(15,8))
plt.hist([df_filtered[df_filtered['gender']=='M']['age'], df_filtered[df_filtered['gender']=='F']['age']], stacked=True, bins=n_bins, label=['Male','Female'])
plt.xlabel('Age')
plt.ylabel('Number of records')
plt.legend()
plot_name = str(plot_path+'sex_age'+".eps")
plt.savefig(plot_name, dpi=150,bbox_inches = "tight")
plt.show()

print(f'Cantidad de Registros: {df_filtered.shape[0]}')
print('Cantidad de Sujetos: {}'.format(np.shape(df_filtered['subject_id'].unique())[0]))

In [ ]:
#df_filtered.to_csv('../df_filtered/ABP_PPG/df_filtered_signals_icu_hadm_sex_age.csv')

<a id='peso_altura'></a>

### Close Conection

In [ ]:
conn.close